<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключаемся-к-созданной-базе-данных" data-toc-modified-id="Подключаемся-к-созданной-базе-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключаемся к созданной базе данных</a></span></li><li><span><a href="#Запросы" data-toc-modified-id="Запросы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Запросы</a></span><ul class="toc-item"><li><span><a href="#Топ-популярных-исполнителей-(чьи-треки-наибоее-часто-слушают-пользователи)" data-toc-modified-id="Топ-популярных-исполнителей-(чьи-треки-наибоее-часто-слушают-пользователи)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Топ популярных исполнителей (чьи треки наибоее часто слушают пользователи)</a></span></li><li><span><a href="#Популярные-исполнители-в-возрастной-категории-18-35" data-toc-modified-id="Популярные-исполнители-в-возрастной-категории-18-35-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Популярные исполнители в возрастной категории 18-35</a></span></li><li><span><a href="#Средняя-продолжительность-треков-по-жанрам" data-toc-modified-id="Средняя-продолжительность-треков-по-жанрам-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Средняя продолжительность треков по жанрам</a></span><ul class="toc-item"><li><span><a href="#жанры-с-наиболее-длительным-композициями" data-toc-modified-id="жанры-с-наиболее-длительным-композициями-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>жанры с наиболее длительным композициями</a></span></li><li><span><a href="#жанры-с-наименее-длительными-композициями" data-toc-modified-id="жанры-с-наименее-длительными-композициями-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>жанры с наименее длительными композициями</a></span></li></ul></li><li><span><a href="#Профили-наиболее-прибыльных-для-сервиса-пользователей-(тех,-кто-приносит-много-денег)" data-toc-modified-id="Профили-наиболее-прибыльных-для-сервиса-пользователей-(тех,-кто-приносит-много-денег)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Профили наиболее прибыльных для сервиса пользователей (тех, кто приносит много денег)</a></span></li><li><span><a href="#Популярность-трека-в-зависимости-от-месяца,-в-который-вышел-альбом" data-toc-modified-id="Популярность-трека-в-зависимости-от-месяца,-в-который-вышел-альбом-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Популярность трека в зависимости от месяца, в который вышел альбом</a></span></li><li><span><a href="#Сводка-по-пользователям-разных-городов" data-toc-modified-id="Сводка-по-пользователям-разных-городов-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Сводка по пользователям разных городов</a></span></li></ul></li></ul></div>

## Подключаемся к созданной базе данных

In [14]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'musician', # имя пользователя
             'pwd': 'music', # пароль
             'host': 'localhost',
             'port': 5433, # порт подключения
             'db': 'music_service_project'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                  db_config['pwd'],
                                                  db_config['host'],
                                                  db_config['port'],
                                                  db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string) 
                       #connect_args={'sslmode':'require'}) 

In [12]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://disk.yandex.ru/i/rZvLUGtkN84StQ")

## Запросы 

### Топ популярных исполнителей (чьи треки наибоее часто слушают пользователи)

In [39]:
query = """ select ar.name, count(tr.track) as "Кол-во прослушиваний"
            from tracks tr
            join artists ar on ar.artist_id = tr.artist_id
            group by ar.name
            order by count(tr.track) desc
            limit 5;
        """

pd.io.sql.read_sql(query, con = engine) 

,name,Кол-во прослушиваний
0,Mansionair,6
1,Веданъ Колодъ,5
2,Fire + Ice,4
3,Devathorn,1
4,Detroit Grand Pubahs,1


### Популярные исполнители в возрастной категории 18-35

In [19]:
query = """ select ar.name, count(tr.track)
            from artists ar
            join tracks tr on ar.artist_id = tr.artist_id
            join users us on tr.user_id = us.user_id
            where us.age between 18 and 35
            group by ar.name
            order by count(tr.track) desc
            limit 10;
        """

pd.io.sql.read_sql(query, con = engine) 

,name,count
0,Mansionair,2
1,Дрыгва,1
2,Freezepop,1
3,Lil Tee Chill Tank Young Buck Brother Mohammed...,1
4,Maïak,1
5,Hachidai Nakamura,1
6,Ministers of Music,1
7,Fire + Ice,1
8,HEALTH,1
9,kreida,1


### Средняя продолжительность треков по жанрам

#### жанры с наиболее длительным композициями

In [34]:
query = """ select g.genre_name, round(avg(tr.duration_sec) / 60, 2) as "Ср. длительность в минутах"
            from genres g
            join tracks tr on g.genre_id = tr.genre_id
            group by g.genre_name
            order by  avg(tr.duration_sec) desc
            limit 5

        """

pd.io.sql.read_sql(query, con = engine) 

,genre_name,Ср. длительность в минутах
0,business,4.63
1,drama,4.44
2,reggae,4.38
3,country,4.27
4,conjazz,4.17


#### жанры с наименее длительными композициями

In [35]:
query = """ select g.genre_name, round(avg(tr.duration_sec) / 60, 2) as "Ср. длительность в минутах"
            from genres g
            join tracks tr on g.genre_id = tr.genre_id
            group by g.genre_name
            order by  avg(tr.duration_sec) 
            limit 5

        """

pd.io.sql.read_sql(query, con = engine) 

,genre_name,Ср. длительность в минутах
0,local,0.77
1,folk,0.95
2,indie,1.02
3,rusrock,1.08
4,latin,1.10


### Профили наиболее прибыльных для сервиса пользователей (тех, кто приносит много денег)

In [41]:
query = """ select us.user_id, us.age, us.city, s.type_of_subscription, p.money_paid
            from users us
            join payments p on us.user_id = p.user_id
            join subscriptions s on p.subscription_id = s.subscription_id
            order by p.money_paid desc
            limit 10

        """

pd.io.sql.read_sql(query, con = engine) 

,user_id,age,city,type_of_subscription,money_paid
0,C3BAA9A9,13,Kazan,plus,4577
1,39DE290E,40,Kazan,plus,4378
2,14FBD1E5,81,Saint-Petersburg,plus,3980
3,842029A1,65,Saint-Petersburg,plus,3582
4,F9AD83A7,66,Moscow,plus,3383
5,ER3489FV,14,Moscow,plus,3184
6,CCD46819,81,Moscow,plus,2985
7,8B14ACB3,45,Kazan,plus,2388
8,BC5A3A29,34,Moscow,plus,2388
9,E1BD98,88,Moscow,plus,2388


### Популярность трека в зависимости от месяца, в который вышел альбом

In [26]:
query = """ select to_char(rel.release_date, 'month') as "month", 
                   round(avg(tr.popularity), 2) as "avg popularity"
            from tracks tr
            join albums al on tr.album_id = al.album_id
            join releases_of_albums rel on al.album_id = rel.album_id
            group by TO_CHAR(rel.release_date, 'month')
            order by avg(tr.popularity) desc

        """

pd.io.sql.read_sql(query, con = engine) 

,month,avg popularity
0,july,78.50
1,august,75.14
2,march,72.00
3,september,72.00
4,may,71.00
5,november,66.71
6,october,65.95
7,december,65.74
8,january,65.60
9,february,63.83


### Сводка по пользователям разных городов

In [38]:
query = """ select us.city, count(us.user_id) as "Кол-во пользователей", 
            round(avg(us.age),2) as "Средний возраст", 
            round(avg(p.money_paid),2) as "Ср. кол-во денег от одного клиента"
            from users us
            join tracks tr on us.user_id = tr.user_id
            join genres g on tr.genre_id = g.genre_id
            join payments p on us.user_id = p.user_id
            group by us.city
            order by count(us.user_id) desc
            
        """

pd.io.sql.read_sql(query, con = engine) 

,city,Кол-во пользователей,Средний возраст,Ср. кол-во денег от одного клиента
0,Moscow,45,58.51,1583.67
1,Saint-Petersburg,30,56.30,1149.10
2,Kazan,13,43.08,1665.08
3,Novosibirsk,4,54.75,2078.75
4,London,1,79.00,0.00
